In [1]:
import tensorflow as tf 


In [2]:
#加载mnist数据集
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

In [3]:
#增加维度
x_train=tf.expand_dims(x_train,-1)
x_test=tf.expand_dims(x_test,-1)

In [4]:
x_train=x_train/255
x_test=x_test/255

In [5]:
x_train=tf.cast(x_train,tf.float32)
x_test=tf.cast(x_test,tf.float32)

In [6]:
dataset_train=tf.data.Dataset.from_tensor_slices((x_train,y_train))
dataset_test=tf.data.Dataset.from_tensor_slices((x_test,y_test))

In [7]:
dataset_train,dataset_test

(<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.uint8)>,
 <TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.uint8)>)

In [8]:
dataset_train=dataset_train.shuffle(x_test.shape[0]).repeat().batch(128)
dataset_test=dataset_test.repeat().batch(128)

In [9]:
model=tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(28,28,1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64,[3,3],activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax')
    ]
)

In [10]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['acc']
)

In [14]:
import  os,datetime
log_dir=os.path.join('logs',datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))

In [24]:
#用于写入自定义变量
file_writer=tf.summary.create_file_writer(log_dir+'/lr')
file_writer.set_as_default()#当使用tf.summary.scalar写入数据的时候会默认调用这个filewriter写入上面那个目录

In [25]:
def lr_schedule(epoch):
    lr=0.2
    if epoch>5:
        lr=0.02
    if epoch>10:
        lr=0.01
    if epoch>20:
        lr=0.005
    tf.summary.scalar('learning_rate',data=lr,step=epoch)
    return lr

In [26]:
#学习速率回调函数，用于动态改变学习速率
lr_callback=tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [ ]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,#每个epoch记录一次
    
)

In [27]:
model.fit(
    dataset_train,
    epochs=25,
    steps_per_epoch=60000//128,
    validation_data=dataset_test,
    validation_steps=10000//128,
    callbacks=[tensorboard_callback,lr_callback ]
)

Train for 468 steps, validate for 78 steps
Epoch 1/25
468/468 [==============================] - 3s 6ms/step - loss: 3.2199 - acc: 0.1047 - val_loss: 2.3198 - val_acc: 0.1029

In [22]:
%reload_ext tensorboard
%matplotlib inline 

In [23]:
%tensorboard --logdir logs

Launching TensorBoard...

In [ ]:
###在命令行中执行tensorboard --logdir logs
